In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from kerastuner.tuners import RandomSearch

# Load data
data = pd.read_csv('AAPL.csv')



ModuleNotFoundError: No module named 'tensorflow'

In [2]:
!pip install tensorflow


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
num_features = data.shape[1]

# Create a copy of the DataFrame without the date column
data_numeric = data.drop('Date', axis=1)

# Normalize data
scaler = MinMaxScaler()
data_norm = scaler.fit_transform(data_numeric)

# Add the date column back to the normalized data
data_norm = np.concatenate((data[['Date']].values, data_norm), axis=1)

# Split data into train and test sets
train_size = int(len(data_norm) * 0.8)
train_data = data_norm[:train_size, :]
test_data = data_norm[train_size:, :]




In [ ]:
# Generate data with sliding windows
def create_dataset(data, num_timesteps):
    X = []
    y = []
    for i in range(num_timesteps, len(data)):
        X.append(data[i-num_timesteps:i, :])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Define the hyperparameters search space
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), 
                   input_shape=(num_timesteps, num_features), 
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                   return_sequences=True))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units', min_value=16, max_value=64, step=16)))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', 
                              values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error')
    return model




In [ ]:
# Perform hyperparameter tuning
num_timesteps = 10

tuner = RandomSearch(build_model, objective='val_loss', max_trials=10, executions_per_trial=3)
X_train, y_train = create_dataset(train_data, num_timesteps)
X_test, y_test = create_dataset(test_data, num_timesteps)
tuner.search(X_train, y_train, epochs=64, batch_size=100, validation_data=(X_test, y_test))



In [ ]:
# Evaluate best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=2)
train_score = best_model.evaluate(X_train, y_train, verbose=0)
print('Train score:', train_score)
test_score = best_model.evaluate(X_test, y_test, verbose=0)
print('Test score:', test_score)




In [ ]:
# Make predictions
train_predictions = best_model.predict(X_train)
test_predictions = best_model.predict(X_test)



In [ ]:
# Inverse normalization of predictions
train_predictions = scaler.inverse_transform(train_predictions)
y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))